In [ ]:
#import string
import re # Regex (expressão regular)
!pip install unidecode
from unicodedata import normalize # utíl para tratamento de texto e compatibilidade
!pip install pandas
import pandas as pd
!pip install nltk
import nltk # Ferramentas de PLN
!pip install sklearn
import sklearn
!pip install seaborn
import seaborn as sns
!pip install matplotlib
import matplotlib.pyplot as plt # Plotagem de gráficos

print("\n" + '\033[1m' + '\033[93m' + "Importações feitas com sucesso!")

## Dados
* As seguintes CSV foram baixados em [Portuguese Tweets for Sentiment Analysis](https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis) criado por [augustop](https://www.kaggle.com/augustop) 

In [ ]:
# https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis
arquivos = [
    'TweetsNeutralNews.csv',
    'TweetsWithTheme.csv'
]

In [ ]:
lista = list()

for arquivo in arquivos:
    df = pd.read_csv(arquivo, sep=",", index_col=None, header=0)
    lista.append(df[['tweet_text','sentiment']])

tweets = pd.concat(lista, axis=0, ignore_index=False)

tweets['score'] = tweets['sentiment'].replace(['Negativo','Neutro', 'Positivo'], [-1, 0, 1])

tweets

In [ ]:
# Limpando variaveis
arquivo = None
arquivos = None
df = None
lista = None

del arquivo
del arquivos
del df
del lista

In [ ]:
# é importante que os dados estejam proximos para não criar um 'Bias' muito grande a um tipo
tweets['sentiment'].value_counts()

In [ ]:
# 'baseline' seria a performance de um classificador tirivial
baseline = tweets['sentiment'].value_counts().max() / tweets['sentiment'].value_counts().sum()

baseline

In [ ]:
print(f'Nosso classificador "inteligente" tem que acertar mais de {int(baseline * 100)}% pra ser melhor que o classificador trivial')

In [ ]:
# Deixando tudo em minúsculo
tweets['tweet_text'] = tweets['tweet_text'].str.lower()

tweets['tweet_text']

In [ ]:
# Removendo tudo entre []
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace='\[.*?\]', value='', regex=True)

tweets['tweet_text']

In [ ]:
# remover links
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace='https?://[A-Za-z0-9./]+', value='', regex=True)
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace='http?://[A-Za-z0-9./]+', value='', regex=True)
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace='wwww?://[A-Za-z0-9./]+', value='', regex=True)
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace='ftp?://[A-Za-z0-9./]+', value='', regex=True)

In [ ]:
# remover emails

In [ ]:
# Removendo números
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace='\d+', value='', regex=True)

tweets['tweet_text']

In [ ]:
# Removendo #rashtags
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace=r'(\#\w+)', value='', regex=True)

tweets['tweet_text']

In [ ]:
# Removendo @Mentions
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace=r'(\@\w+)', value='', regex=True)

tweets['tweet_text']

In [ ]:
# Removendo a pontuação (EX: "!?.,/|#$%¨&")
tweets['tweet_text'] = tweets['tweet_text'].replace(to_replace=r'[^\w\s]', value='', regex=True)

tweets['tweet_text']

In [ ]:
# Removendo acentuação
tweets['tweet_text'] = tweets['tweet_text'].str.normalize('NFKD').str.encode('ASCII', errors='ignore').str.decode('UTF-8')

tweets['tweet_text']

In [ ]:
# Uma stopword pode ser considerada uma palavra irrelevante para a análise
nltk.download('stopwords')
# RSLP(Removedor de Sufixos da Língua Portuguesa)
nltk.download('rslp')

In [ ]:
# Criando uma lista de stopWords
stopWords = set(nltk.corpus.stopwords.words('portuguese'))
# O processo de stemming consiste em reduzir a palavra à sua raiz (sem levar em conta a classe gramatical)
stemmer = nltk.stem.RSLPStemmer() # RSLP(Removedor de Sufixos da Língua Portuguesa)
# Separa por tokens (tokens = palavras)
tokenizer = nltk.tokenize.WordPunctTokenizer()

In [ ]:
todasPalavras = ' '.join([text for text in tweets['tweet_text']])

In [ ]:
tokenizer = nltk.tokenize.WordPunctTokenizer()

tokensPalavras = tokenizer.tokenize(todasPalavras)

In [ ]:
# Removendo StopWord e letras soltas do nosso texto
todasPalavrasFiltradas = [palavra for palavra in tokensPalavras if palavra not in stopWords and len(palavra) > 1]

todasPalavrasFiltradas

In [ ]:
# Nesse passo é criado uma função que faz a Tokenização e o Stemming
def criarBagOfWords(text, stop_words=stopWords):
    tokens = tokenizer.tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words and len(token) > 1]
    return ' '.join(tokens)

tweets['bag_of_words'] = tweets['tweet_text'].apply(criarBagOfWords)

In [ ]:
# Plotar as 10 palavras mais frequentes

plotar = False # Precisa ser True para plotar

if(plotar):
    todasPalavras = ' '.join([text for text in tweets['bag_of_words']])
    tokensPalavras = tokenizer.tokenize(todasPalavras)
    frequenciaPalavras = nltk.FreqDist(todasPalavrasFiltradas)
    df_frequencia_palavras = pd.DataFrame({
        'palavra': list(frequenciaPalavras.keys()),
        'frequencia': list(frequenciaPalavras.values())
    })
    maisFrequentes = df_frequencia_palavras.nlargest(columns='frequencia', n=10)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data=maisFrequentes, x='palavra', y='frequencia')
    ax.set(ylabel='frequencia')
    plt.show()

In [ ]:
tweets.head()

In [ ]:
tweets.tail()